# data extaction

In [3]:
from math import log
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [4]:
# get the data
filename = 'train.csv'
label_map = ['Not Survived', 'Survived']
names=['Survived', 'Pclass', 'Name', 'Gender', 'Age', 'SibSp', 'Parch', 'Embarked']
df=pd.read_csv('train.csv',names=names, na_filter=False)
df.head(10)

,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Embarked
0,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,S
6,0,3,"Moran, Mr. James",male,,0,0,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,S


In [5]:
def normalizeAge(age):
    if age == '':
        return 3
    age = int(float(age))
    if age <= 2:
        return 0
    elif age <= 13:
        return 1
    elif age <= 24:
        return 2
    elif age <= 50:
        return 3
    return 4

In [6]:
def normalizeEmbarked(embarked):
    embarked = embarked[0]
    if embarked == 'S':
        return 0
    elif embarked == 'Q':
        return 1
    return 2

In [10]:
def getData():
    global X
    X = []
    first = True
    i = 0
    for line in open(filename):
        if first:
            first = False
        else:
            row = line.split(',')
            assert(len(row) == 9)
            X.append(list())
            
            # Survived (index: 0)
            X[i].append(int(row[0]))
            # Pclass (index: 1)
            X[i].append(int(row[1]) - 1)
            # Gender (index: 2)
            X[i].append(0) if row[4] == 'male' else X[i].append(1)
            # Age (index: 3)
            X[i].append(normalizeAge(row[5]))
            # SibSP (index: 4)
            X[i].append(int(row[6]))
            # ParCh (index: 5)
            X[i].append(int(row[7]))
            # Embarked (index: 6)
            X[i].append(normalizeEmbarked(row[8]))
            i = i + 1

In [11]:
getData()

In [12]:
survived = 0
notSurvived = 0
for i in range(len(X)):
    if X[i][0] == 0:
        notSurvived += 1
    else:
        survived += 1

In [13]:
def calPxi(index, value):
    cnt = 0
    for i in range(len(X)):
        if X[i][index] == value:
            cnt += 1
    return cnt / len(X)

In [474]:
def calHxi(index):
    result = 0
    for i in range(10):
        Pxi = calPxi(index, i)
        if Pxi > 0:
            result -= Pxi * log(Pxi) / log(2)
    return result

In [475]:
def calPxy(xFactorIndex, xValue, yValue):
    cnt = 0
    for i in range(len(X)):
        if X[i][0] == yValue and X[i][xFactorIndex] == xValue:
            cnt += 1
    return cnt / len(X) 

In [476]:
def calHxy(xFactorIndex):
    result = 0
    for i in range(10):
        for j in range(2):
            Pxy = calPxy(xFactorIndex, i, j)
            if Pxy > 0:
                result -= Pxy * log(Pxy) / log(2)
    return result

In [477]:
def calIPClass():
    return calHxi(1) + calHxi(0) - calHxy(1)

In [478]:
def calIGender():
    return calHxi(2) + calHxi(0) - calHxy(2)

In [479]:
def calIAge():
    return calHxi(3) + calHxi(0) - calHxy(3)

In [480]:
def calISibSp():
    return calHxi(4) + calHxi(0) - calHxy(4)

In [481]:
def calIParch():
    return calHxi(5) + calHxi(0) - calHxy(5)

In [482]:
def calIEmbark():
    return calHxi(6) + calHxi(0) - calHxy(6)

In [485]:
print("I value for PClass:", calIPClass())
print("I value for Gender:", calIGender())
print("I value for Age:", calIAge())
print("I value for SibSp:", calISibSp())
print("I value for Parch:", calIParch())
print("I value for Embark:", calIEmbark())

I value for PClass: 0.08383104529601138
I value for Gender: 0.21766010666061408
I value for Age: 0.011278308150915795
I value for SibSp: 0.03346632133870964
I value for Parch: 0.023610547633470702
I value for Embark: 0.02214740635533996
